In [2]:
%matplotlib inline
import numpy as np
np.random.seed(2016)
import matplotlib.pyplot as plt

import os
import glob
import datetime
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from scipy.misc import imread
from scipy.misc import imresize

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras import __version__ as keras_version
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
def load_cropped_train():
    X_train = []
    X_train_id = []
    y_train = []
    start_time = time.time()

    print('Read train images')
    folders = ['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join('cropped_data', fld, '*.jpg')
        files = sorted(glob.glob(path))
        for fl in files:
            flbase = os.path.basename(fl)
            img = image.load_img(fl, target_size=(299, 299))
            img = image.img_to_array(img)
            X_train.append(img)
            X_train_id.append(fld + '/' + flbase)
            y_train.append(index)

    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, y_train, X_train_id

def read_and_normalize_cropped_train_data():
    train_data, train_target, train_id = load_cropped_train()

    print('Convert to numpy...')
    train_data = np.array(train_data)
    train_target = np.array(train_target)

    print('Convert to float...')
    train_data = train_data.astype('float32')
    train_data = train_data / 255
    train_target = np_utils.to_categorical(train_target, 8)

    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target, train_id

In [3]:
train_data, train_target, train_id = read_and_normalize_cropped_train_data()

Read train images
Load folder ALB (Index: 0)
Load folder BET (Index: 1)
Load folder DOL (Index: 2)
Load folder LAG (Index: 3)
Load folder OTHER (Index: 4)
Load folder SHARK (Index: 5)
Load folder YFT (Index: 6)
Read train data time: 27.69 seconds
Convert to numpy...
Convert to float...
('Train shape:', (4371, 299, 299, 3))
(4371, 'train samples')


In [3]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(8, activation='softmax')(x)
# this is the model we will train
model = Model(input=base_model.input, output=predictions)
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='sgd', loss='categorical_crossentropy')

In [19]:
batch_size = 16
nb_epoch = 30
random_state = 51

X_train, X_test, y_train, y_test = train_test_split(train_data, train_target, test_size=0.2, random_state=0)

In [20]:
callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),]
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, shuffle=True,
          verbose=2, validation_data=(X_test, y_test), callbacks=callbacks)

Train on 3496 samples, validate on 875 samples
Epoch 1/30
75s - loss: 0.6114 - val_loss: 0.6613
Epoch 2/30
75s - loss: 0.5875 - val_loss: 0.6538
Epoch 3/30
75s - loss: 0.5648 - val_loss: 0.6752
Epoch 4/30
75s - loss: 0.5422 - val_loss: 0.6934
Epoch 5/30
75s - loss: 0.5424 - val_loss: 0.7287
Epoch 6/30
75s - loss: 0.5299 - val_loss: 0.5913
Epoch 7/30
75s - loss: 0.5018 - val_loss: 0.6690
Epoch 8/30
75s - loss: 0.4825 - val_loss: 0.6614
Epoch 9/30
75s - loss: 0.4722 - val_loss: 0.5614
Epoch 10/30
75s - loss: 0.4808 - val_loss: 0.5637
Epoch 11/30
75s - loss: 0.4686 - val_loss: 0.7041
Epoch 12/30
75s - loss: 0.4533 - val_loss: 0.6013
Epoch 13/30
75s - loss: 0.4507 - val_loss: 0.5387
Epoch 14/30
75s - loss: 0.4348 - val_loss: 0.5472
Epoch 15/30
75s - loss: 0.4074 - val_loss: 0.5306
Epoch 16/30
75s - loss: 0.3952 - val_loss: 0.5555
Epoch 17/30
75s - loss: 0.4007 - val_loss: 0.6027
Epoch 18/30
75s - loss: 0.3822 - val_loss: 0.5369
Epoch 19/30
75s - loss: 0.3821 - val_loss: 0.5544


In [49]:
predictions_valid = model.predict(X_train.astype('float32'), batch_size=batch_size, verbose=2)
score = log_loss(y_train, predictions_valid)

In [50]:
print "log loss score: ", score

log loss score:  0.285989247257


In [39]:
from sklearn.metrics import accuracy_score
y_true = np.argmax(y_test, axis = 1)
y_pred = np.argmax(predictions_valid, axis = 1)
acc = accuracy_score(y_true, y_pred, normalize=True, sample_weight=None)

In [46]:
print "accuracy: ", acc

 accuracy:  0.813714285714


In [47]:
print y_true[0:100]
print y_pred[0:100]

[0 5 0 6 0 0 0 0 6 0 0 6 6 0 0 0 0 6 0 6 0 6 0 6 0 0 5 6 0 0 0 0 0 1 0 0 1
 0 1 4 0 0 0 0 6 0 2 0 0 0 1 0 0 0 3 6 6 6 6 4 6 0 1 0 0 0 6 0 0 6 6 0 0 0
 0 1 6 0 0 0 0 6 0 0 0 4 0 3 0 0 0 0 6 0 6 0 6 0 0 0]
[0 5 0 2 0 0 0 6 0 0 0 6 6 0 0 0 0 6 0 6 0 6 0 6 0 0 5 6 0 6 0 0 0 1 4 0 0
 0 6 4 0 0 0 0 6 0 2 0 0 0 1 4 0 0 3 1 6 6 6 4 6 0 1 0 0 1 0 0 0 0 4 0 1 0
 0 0 6 0 1 0 0 6 6 0 0 4 0 3 4 6 0 0 6 0 6 0 6 0 0 0]
